# Data Wrangling

In [1]:
import pandas as pd
import aqi
import pycountry_convert as pc
import pycountry
import numpy as np

In [2]:
data = pd.read_csv("../data/raw/world_air_quality.csv", sep=';')

data = data.drop(["City", "Location", "Source Name"], axis=1)

data = data.rename(columns={'Last Updated': 'time', 
                     'Country Label': 'countryname',
                     'Value': 'value',
                     'Pollutant': 'pollutant',
                     'Coordinates': 'coordinates',
                     'Unit': 'unit',
                     'Country Code': 'country_code'})

data['time'] = pd.to_datetime(data['time']).dt.date
data.head()

,country_code,coordinates,pollutant,unit,value,time,countryname
0,JP,"33.880833, 130.873056",NO,ppm,0.002,2024-03-10,Japan
1,JP,"33.898056, 130.81",NO2,ppm,0.005,2024-03-10,Japan
2,JP,"33.895833, 130.935833",NOX,ppm,0.013,2024-03-10,Japan
3,JP,"42.786944, 141.605",NO2,ppm,0.004,2024-03-10,Japan
4,JP,"35.653889, 140.097778",NOX,ppm,0.003,2024-03-10,Japan


In [3]:
# Create continent column
def get_continent(country_code):
    try:
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except KeyError:
        return 'Antartica'

data['continent'] = data.apply(lambda row: get_continent(row['country_code']), axis=1)
data


,country_code,coordinates,pollutant,unit,value,time,countryname,continent
0,JP,"33.880833, 130.873056",NO,ppm,0.002,2024-03-10,Japan,Asia
1,JP,"33.898056, 130.81",NO2,ppm,0.005,2024-03-10,Japan,Asia
2,JP,"33.895833, 130.935833",NOX,ppm,0.013,2024-03-10,Japan,Asia
3,JP,"42.786944, 141.605",NO2,ppm,0.004,2024-03-10,Japan,Asia
4,JP,"35.653889, 140.097778",NOX,ppm,0.003,2024-03-10,Japan,Asia
...,...,...,...,...,...,...,...,...
54250,IT,"40.64389299999999, 15.872893000000001",CO,µg/m³,295.000,2024-03-11,Italy,Europe
54251,IT,"46.030833, 11.905833",O3,µg/m³,35.000,2024-03-11,Italy,Europe
54252,IT,"43.55472, 10.32972",CO,µg/m³,600.000,2024-03-11,Italy,Europe
54253,IT,"43.91611, 11.006939999999998",PM2.5,µg/m³,15.000,2024-03-11,Italy,Europe


In [4]:
data[data['countryname'].isnull()]['country_code'].unique()


array(['XK', 'AJ', 'CE', 'UC', 'CW', 'TI', 'IZ', 'KU', 'BK', 'VM', 'AQ',
       'TX', 'KV'], dtype=object)

In [5]:
# Impute missing country names from country code
data.loc[(data['country_code'] == 'XK') & (data['countryname'].isnull()), 'countryname'] = 'Kosovo'
data.loc[(data['country_code'] == 'AJ') & (data['countryname'].isnull()), 'countryname'] = 'Azerbaijan'
data.loc[(data['country_code'] == 'CE') & (data['countryname'].isnull()), 'countryname'] = 'Sri Lanka'
data.loc[(data['country_code'] == 'UC') & (data['countryname'].isnull()), 'countryname'] = 'Curacao'
data.loc[(data['country_code'] == 'TI') & (data['countryname'].isnull()), 'countryname'] = 'Tajikistan'
data.loc[(data['country_code'] == 'IZ') & (data['countryname'].isnull()), 'countryname'] = 'Iraq'
data.loc[(data['country_code'] == 'KU') & (data['countryname'].isnull()), 'countryname'] = 'Kuwait'
data.loc[(data['country_code'] == 'BK') & (data['countryname'].isnull()), 'countryname'] = 'Bosnia-Herzegovina'
data.loc[(data['country_code'] == 'VM') & (data['countryname'].isnull()), 'countryname'] = 'Vietnam'
data.loc[(data['country_code'] == 'TX') & (data['countryname'].isnull()), 'countryname'] = 'Turkmenistan'
data.loc[(data['country_code'] == 'KV') & (data['countryname'].isnull()), 'countryname'] = 'Kosovo'
data.loc[(data['country_code'] == 'SU') & (data['countryname'].isnull()), 'countryname'] = 'Sudan'
data.loc[(data['country_code'] == 'CS') & (data['countryname'].isnull()), 'countryname'] = 'Costa Rica'
data.loc[(data['country_code'] == 'CW') & (data['countryname'].isnull()), 'countryname'] = 'Cook Islands'
data.loc[(data['country_code'] == 'AQ') & (data['countryname'].isnull()), 'countryname'] = 'Antartica'

# Check for missing names
data[data['countryname'].isnull()]['country_code'].unique()


array([], dtype=object)

In [6]:
pollutants = ['NO', 'NO2', 'NOX', 'SO2', 'PM2.5', 'CO', 'O3', 'PM10', 'PM1']

#Engineer the Overall AQI Feature 
def aqi_calculation(C, breakpoints):
    for C_low, C_high, I_low, I_high in breakpoints:
        if C_low <= C <= C_high:
            return ((I_high - I_low) / (C_high - C_low)) * (C - C_low) + I_low
    return np.nan
#AQI breakpoints, based on the US EPA System
    #https://aqs.epa.gov/aqsweb/documents/codetables/aqi_breakpoints.html
breakpoints = {
    'PM2.5': [(0.0, 12.0, 0, 50), (12.1, 35.4, 51, 100), (35.5, 55.4, 101, 150), (55.5, 150.4, 151, 200), (150.5, 250.4, 201, 300), (250.5, 350.4, 301, 400), (350.5, 500.4, 401, 500)],
    'PM10': [(0, 54, 0, 50), (55, 154, 51, 100), (155, 254, 101, 150), (255, 354, 151, 200), (355, 424, 201, 300), (425, 504, 301, 400), (505, 604, 401, 500)],
    'CO': [(0.0, 4.4, 0, 50), (4.5, 9.4, 51, 100), (9.5, 12.4, 101, 150), (12.5, 15.4, 151, 200), (15.5, 30.4, 201, 300), (30.5, 40.4, 301, 400), (40.5, 50.4, 401, 500)],
    'SO2': [(0, 35, 0, 50), (36, 75, 51, 100), (76, 185, 101, 150), (186, 304, 151, 200), (305, 604, 201, 300), (605, 804, 301, 400), (805, 1004, 401, 500)],
    'NO2': [(0, 53, 0, 50), (54, 100, 51, 100), (101, 360, 101, 150), (361, 649, 151, 200), (650, 1249, 201, 300), (1250, 1649, 301, 400), (1650, 2049, 401, 500)],
    'O3': [(0.125, 0.164, 101, 150), (0.165, 0.204, 151, 200), (0.205, 0.404, 201, 300), (0.405, 0.504, 301, 400), (0.505, 0.604, 401, 500)],
    'O3_8hr': [(0.0, 0.054, 0, 50), (0.055, 0.07, 51, 100), (0.071, 0.085, 101, 150), (0.086, 0.105, 151, 200)]
}

#Calculate AQI based on pollutation present in each row
def calculate_row_aqi(row):
    pollutant = row['pollutant']
    concentration = row['value']
    if pollutant in breakpoints:
        return aqi_calculation(concentration, breakpoints[pollutant])
    else:
        return np.nan

data['AQI'] = data.apply(calculate_row_aqi, axis=1)

data

,country_code,coordinates,pollutant,unit,value,time,countryname,continent,AQI
0,JP,"33.880833, 130.873056",NO,ppm,0.002,2024-03-10,Japan,Asia,NaN
1,JP,"33.898056, 130.81",NO2,ppm,0.005,2024-03-10,Japan,Asia,0.004717
2,JP,"33.895833, 130.935833",NOX,ppm,0.013,2024-03-10,Japan,Asia,NaN
3,JP,"42.786944, 141.605",NO2,ppm,0.004,2024-03-10,Japan,Asia,0.003774
4,JP,"35.653889, 140.097778",NOX,ppm,0.003,2024-03-10,Japan,Asia,NaN
...,...,...,...,...,...,...,...,...,...
54250,IT,"40.64389299999999, 15.872893000000001",CO,µg/m³,295.000,2024-03-11,Italy,Europe,NaN
54251,IT,"46.030833, 11.905833",O3,µg/m³,35.000,2024-03-11,Italy,Europe,NaN
54252,IT,"43.55472, 10.32972",CO,µg/m³,600.000,2024-03-11,Italy,Europe,NaN
54253,IT,"43.91611, 11.006939999999998",PM2.5,µg/m³,15.000,2024-03-11,Italy,Europe,57.098712


In [7]:
correction_mapping = {
    "Czech Republic": "Czechia",
    "Russian Federation": "Russia",
    "Macedonia, The former Yugoslav Rep. of": "North Macedonia",
    "Taiwan, China": "Taiwan",
    "United States": "United States of America",
    "Lao People's Dem. Rep.": "Laos",
    "Moldova, Republic of": "Moldova",
    "Serbia": "Republic of Serbia",
    "Hong Kong, China": "China",  # Assuming you want to map Hong Kong to China 
    "West Bank and Gaza Strip": "Palestine",  # Assuming mapping to Palestine
    "Viet Nam": "Vietnam"
}

data['countryname'] = data['countryname'].apply(lambda x: correction_mapping.get(x, x))

countries_to_drop = ['Andorra', 'Gibraltar', 'Malta']
filtered_data = data[~data['countryname'].isin(countries_to_drop)]

data = filtered_data

data = data.sort_values(by='countryname')
data

,country_code,coordinates,pollutant,unit,value,time,countryname,continent,AQI
46339,AF,"34.554384, 69.204597",PM2.5,µg/m³,136.000000,2021-08-15,Afghanistan,Asia,192.564805
20436,AF,"34.535812, 69.190514",PM2.5,µg/m³,-999.000000,2021-08-16,Afghanistan,Asia,NaN
41831,DZ,"36.755798, 3.039114",PM2.5,µg/m³,78.000000,2024-03-25,Algeria,Africa,162.617492
49813,AQ,"-74.16627777777778, 4.577805599999999",NO2,ppm,0.051045,2021-02-04,Antartica,Antartica,0.048155
49812,AQ,"-74.16627777777778, 4.577805599999999",CO,ppm,2.105187,2021-02-04,Antartica,Antartica,23.922580
...,...,...,...,...,...,...,...,...,...
6286,VN,"21.02177, 105.819002",PM2.5,µg/m³,12.900000,2016-11-09,Vietnam,Asia,52.682403
20327,VM,"10.782773, 106.700035",PM2.5,µg/m³,-999.000000,2023-05-31,Vietnam,Antartica,NaN
18437,VN,"21.048, 105.8",PM2.5,µg/m³,36.900000,2015-08-21,Vietnam,Asia,104.447236
34381,VM,"21.021939, 105.818806",PM2.5,µg/m³,36.000000,2023-05-31,Vietnam,Antartica,102.231156


In [8]:
# Save processed data to file
data.to_csv("../data/processed/world_air_quality.csv")

In [9]:
data

,country_code,coordinates,pollutant,unit,value,time,countryname,continent,AQI
46339,AF,"34.554384, 69.204597",PM2.5,µg/m³,136.000000,2021-08-15,Afghanistan,Asia,192.564805
20436,AF,"34.535812, 69.190514",PM2.5,µg/m³,-999.000000,2021-08-16,Afghanistan,Asia,NaN
41831,DZ,"36.755798, 3.039114",PM2.5,µg/m³,78.000000,2024-03-25,Algeria,Africa,162.617492
49813,AQ,"-74.16627777777778, 4.577805599999999",NO2,ppm,0.051045,2021-02-04,Antartica,Antartica,0.048155
49812,AQ,"-74.16627777777778, 4.577805599999999",CO,ppm,2.105187,2021-02-04,Antartica,Antartica,23.922580
...,...,...,...,...,...,...,...,...,...
6286,VN,"21.02177, 105.819002",PM2.5,µg/m³,12.900000,2016-11-09,Vietnam,Asia,52.682403
20327,VM,"10.782773, 106.700035",PM2.5,µg/m³,-999.000000,2023-05-31,Vietnam,Antartica,NaN
18437,VN,"21.048, 105.8",PM2.5,µg/m³,36.900000,2015-08-21,Vietnam,Asia,104.447236
34381,VM,"21.021939, 105.818806",PM2.5,µg/m³,36.000000,2023-05-31,Vietnam,Antartica,102.231156
